# Image Matching and Homography Estimation with OpenCV and LightGlue

In [1]:
import os
import cv2 
import time
import json
import math
import torch
import numpy as np
from vidstab import VidStab
import matplotlib.pyplot as plt

from lightglue import viz2d
from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, rbd, load_image_from_path
import CSRansac

from vidstab import VidStab

In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 'mps', 'cpu'

extractor = SuperPoint(max_num_keypoints=2048).eval().to(device)  # load the extractor
#matcher = LightGlue(features='superpoint', depth_confidence=0.9, width_confidence=0.95).eval().to(device)
matcher = LightGlue(features='superpoint', depth_confidence=-1, width_confidence=-1).eval().to(device)
#matcher.compile(mode='reduce-overhead')

In [4]:
print(torch.__version__)
print(device)

2.1.2
cuda


In [5]:
def match_lightglue(img0, img1):
    img0 = load_image(img0)
    img1 = load_image(img1)

    # extract local features
    feats0 = extractor.extract(img0.to(device))  # auto-resize the image, disable with resize=None
    feats1 = extractor.extract(img1.to(device))
    
    # match the features
    matches01 = matcher({'image0': feats0, 'image1': feats1})
    feats0, feats1, matches01 = [rbd(x) for x in [feats0, feats1, matches01]]  # remove batch dimension
    
    # get results
    kpts0 = feats0["keypoints"]
    kpts1 = feats1["keypoints"]
    matches = matches01['matches']  # indices with shape (K,2)
    points0 = kpts0[matches[..., 0]]  # coordinates in img0, shape (K,2)
    points1 = kpts1[matches[..., 1]]  # coordinates in img1, shape (K,2)
        
    return {
        "points0": points0,
        "points1": points1,
    }

## Dataset 전처리

In [6]:
aircraft_datasets = "D:/aircraft_datasets"

lables = os.path.join(aircraft_datasets + "/label")

In [12]:
# 원본 좌표값과 실수형 좌표값을 불러옴
import copy

origin_coordinate = []
float_origin_coordinate = []

# 원점 좌표값 불러오기
for label_file in os.listdir(lables):
    label_path = os.path.join(lables, label_file)
    with open(label_path, "r") as f:
        json_file = json.load(f)
        coord = json_file["targetAnnotation"]
        float_coord = copy.deepcopy(coord)
        float_origin_coordinate.append(float_coord)
        
        coord[0] = round(coord[0] * 640, 4)
        coord[1] = round(coord[1] * 480, 4)
        origin_coordinate.append(coord)

print(origin_coordinate)
print(len(origin_coordinate))

print(float_origin_coordinate)
print(len(float_origin_coordinate))
print(type(float_origin_coordinate[0][0]))


[[319.172, 270.552], [320.0, 265.245], [344.465, 256.029], [313.576, 257.296], [325.482, 168.084], [315.94, 202.489], [325.479, 168.08], [312.391, 306.427], [320.0, 265.239], [331.487, 26.903], [316.523, 203.088], [329.478, 59.023], [320.0, 337.576], [324.136, 161.36], [309.347, 253.744], [321.263, 248.873], [332.852, 236.023], [326.048, 203.802], [318.489, 251.06], [320.965, 255.826], [321.255, 215.706], [319.453, 225.752], [319.453, 180.869], [321.201, 215.638], [321.228, 215.672], [316.375, 230.084], [316.206, 231.433], [320.898, 312.286], [320.951, 198.621], [315.928, 231.5], [320.895, 257.614], [320.822, 257.477], [320.821, 257.478], [320.653, 290.011], [320.729, 257.293], [320.0, 291.92], [320.0, 257.736], [320.0, 485.359], [318.014, 279.459], [314.676, 328.529]]
40
[[0.4987062, 0.563651], [0.5, 0.5525945], [0.5382264, 0.5333939999999999], [0.4899627, 0.5360328999999999], [0.5085652, 0.3501746], [0.4936557, 0.42185229999999996], [0.5085613, 0.3501674], [0.4881113, 0.6383890999999

In [14]:
video_dir = os.path.join(aircraft_datasets, "video")
target_image_dir = os.path.join(aircraft_datasets, "target_image")
# output_dir = os.path.join(aircraft_datasets, "frames_from_video")
# stabilized_frame_path = os.path.join(aircraft_datasets, "stabilized_frame")
stabilizer = VidStab()

In [15]:
#원본 이미지를 기준으로 호모그래피 행렬을 구하고, 호모그래피 행렬을 이용하여 특징점의 좌표를 변환하는 코드
len_coord = len(origin_coordinate)

coord_list = [[] for _i in range(len(origin_coordinate))]

target_images = []
for image_file in os.listdir(target_image_dir):
    image_path = os.path.join(target_image_dir, image_file)
    target_images.append(image_path)

# 에러를 저장할 리스트
disappear_errors = []
misannotate_errors = []
pixel_errors = []

missing_inlier = 0
failed_inliers = 0

['D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00001.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00002.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00003.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00004.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00005.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00006.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00007.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00008.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00009.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00010.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00011.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00012.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00013.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00014.png', 'D:/aircraft_datasets\\target_image\\FuelPumpRemoval_00015.png', 'D:/aircraft_datasets\\t

In [ ]:
# 에러 측정을 위해 총 10번 반복
for i in range(10):
    for i in range(len_coord):
        target_image = target_images[i]
        len_target_image = len(target_images)
        
        x = origin_coordinate[i][0]
        y = origin_coordinate[i][1]
        
        target_image = load_image(target_image, grayscale=True)
        cap = cv2.VideoCapture('demo_video_resized.mp4')

        # 각 프레임 처리
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            feats0 = extractor.extract(image0.to(device))
            image1 = stabilizer.stabilize_frame(input_frame = frame)
            image1 = load_image(frame, grayscale=True)
            feats1 = extractor.extract(image1.to(device))
            
            matches01 = matcher({"image0": feats0, "image1": feats1})
            
            feats0, feats1, matches01 = [
                rbd(x) for x in [feats0, feats1, matches01]
            ]  # remove batch dimension
            
            kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
            m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
            
            homography, _ = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
            projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
            
            
        cap.release()
            
        

In [ ]:
# 10번 반복하여 측정한 에러를 구함
for k in range(10):
    # 좌표의 개수(동영상의 개수)만큼 반복
    for i in range(len_coord):
        target_image = target_images[i]
        len_target_image = len(target_images)
        x = origin_coordinate[i][0]
        y = origin_coordinate[i][1]
        
        # 두 번째 차원의 리스트 초기화
        coord_list[i] = [[] for _ in range(_len_images)]

        img0 = _images[0] # 첫 번째 이미지를 target 이미지로 설정
        for j in range(_len_images):
            if j != _len_images - 1:
                img1 = _images[j+1]

                # LightGlue
                results_lightglue = match_lightglue(img0, img1)
                target_keypoint = results_lightglue["points0"].cpu().numpy()
                frame_keypoint = results_lightglue["points1"].cpu().numpy()
                if len(target_keypoint) < 6:
                    missing_inlier += 1

                homography, mask = CSRansac.csransac(target_keypoint, frame_keypoint)
                if mask == 0.3:
                    failed_inliers += 1
                projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)

                coord_list[i][j].append(projected_pts)
                
    #에러 측정            
    disappear_error = 0
    misannotate_error = 0
    pixel_error = 0

    for i in range(len_coord):
        origin_x = float_origin_coordinate[i][0]
        origin_y = float_origin_coordinate[i][1]
        
        for j in range(len(coord_list[i])-1):
            _coord = coord_list[i][j]
            
            x = _coord[0][0]
            y = _coord[0][1]
            
            x = x / 640
            y = y / 480
            
            x = round(x, 4)
            y = round(y, 4)
            
            # disappear_error
            if x < 0 or x > 1 or y < 0 or y > 1:
                disappear_error += 1
            
            distance = math.sqrt((origin_x - x)**2 + (origin_y - y)**2)
            
            # num_error
            if distance > 0.1:
                misannotate_error += 1
            
            # pixel_error
            if distance > pixel_error:
                pixel_error = distance
               
    print("disappear_error:", disappear_error)
    print("misannotate_error:", misannotate_error)
    print("pixel_error:", pixel_error)

    disappear_errors.append(disappear_error)
    misannotate_errors.append(misannotate_error)
    pixel_errors.append(pixel_error)
    

print("missing_inlier:", missing_inlier)
print("failed_inliers:", failed_inliers)

## Compare Homography Matrices

In [31]:
#원본 이미지를 기준으로 호모그래피 행렬을 구하고, 호모그래피 행렬을 이용하여 특징점의 좌표를 변환하는 코드
len_coord = len(origin_coordinate)

coord_list = [[] for _i in range(len(origin_coordinate))]

for i in range(len_coord):
    _images = images[i]
    _len_images = len(_images)
    x = origin_coordinate[i][0]
    y = origin_coordinate[i][1]
    
    # 두 번째 차원의 리스트 초기화
    coord_list[i] = [[] for _ in range(_len_images)]

    img0 = _images[0]
    for j in range(_len_images):
        if j != _len_images - 1:
            img1 = _images[j+1]

            # LightGlue
            results_lightglue = match_lightglue(img0, img1, cfg.lightglue)
            target_keypoint = results_lightglue["points0"].cpu().numpy()
            frame_keypoint = results_lightglue["points1"].cpu().numpy()

            homography, _ = CSRansac.csransac(target_keypoint, frame_keypoint)
            projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)

            coord_list[i][j].append(projected_pts)

In [34]:
# 각 행의 이름을 지정하기 위한 딕셔너리 생성
data_dict = {}
for i, row in enumerate(coord_list):
    key = f"video_{i + 1}"  # 각 행의 이름 생성 (row_1, row_2, ...)
    data_dict[key] = row

filename = "test_coord_list.json"
file_path = os.path.join(aircraft_datasets, filename)
with open(file_path, "w") as f:
    json.dump(data_dict, f, indent=4)

In [24]:
#안정화된 이미지를 기준으로 호모그래피 행렬을 구하고, 호모그래피 행렬을 이용하여 특징점의 좌표를 변환하는 코드
len_coord = len(origin_coordinate)

stable_coord_list = [[] for _i in range(len(origin_coordinate))]

for i in range(len_coord):
    _stablized_images = stablized_images[i]
    _len_stablized_images= len(_stablized_images)
    x = origin_coordinate[i][0]
    y = origin_coordinate[i][1]
    
    stable_coord_list[i] = [[] for _j in range(_len_stablized_images)]
    
    img0 = _stablized_images[0]
    for j in range(_len_stablized_images):
        if j != _len_stablized_images - 1:
            img1 = _stablized_images[j+1]
        
            # LightGlue
            results_lightglue = match_lightglue(img0, img1, cfg.lightglue)
            target_keypoint = results_lightglue["points0"].cpu().numpy()
            frame_keypoint = results_lightglue["points1"].cpu().numpy()
            
            homography, _ = CSRansac.csransac(target_keypoint, frame_keypoint)
            projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
            
            stable_coord_list[i][j].append(projected_pts)

In [27]:
# 각 행의 이름을 지정하기 위한 딕셔너리 생성
stabilized_data_dict = {}
for i, row in enumerate(stable_coord_list):
    key = f"video_{i + 1}"  # 각 행의 이름 생성 (row_1, row_2, ...)
    stabilized_data_dict[key] = row

filename = "test_stabilized_coord_list.json"
file_path = os.path.join(aircraft_datasets, filename)
with open(file_path, "w") as f:
    json.dump(stabilized_data_dict, f, indent=4)

In [10]:
# #프레임을 동영상으로 만드는 코드

# # 폴더 경로 설정
# folder = 'video'

# # 동영상 저장 경로 설정
# output_video_path = 'result_origin.mp4'

# # 동영상 속성 설정
# fourcc = cv2.VideoWriter_fourcc(*'XVID')  # 코덱 설정 (XVID를 사용하면 AVI 형식으로 저장)
# fps = 30.0  # 초당 프레임 수
# frame_width = 640  # 프레임 너비
# frame_height = 480  # 프레임 높이

# out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# x = 637 // 2
# y = 367 // 2

# i = 0
# for name in os.listdir(folder):
#     img = cv2.imread(os.path.join(folder, name))
#     if i == 0:
#         cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
#     elif i == len_img - 1:
#         break
#     else:
#         x = round(coord_list[i][0])
#         y = round(coord_list[i][1])
#         cv2.circle(img, (x, y), 3, (0, 0, 255), -1)

#     i = i + 1
    
#     # 프레임을 동영상에 추가
#     out.write(img)

# # 동영상 저장 종료
# out.release()


## Error Estimate

In [11]:
#원본 이미지를 기준으로 호모그래피 행렬을 구하고, 호모그래피 행렬을 이용하여 특징점의 좌표를 변환하는 코드
len_coord = len(origin_coordinate)

coord_list = [[] for _i in range(len(origin_coordinate))]

disappear_errors = []
misannotate_errors = []
pixel_errors = []

missing_inlier = 0
failed_inliers = 0

# 10번 반복하여 측정한 에러를 구함
for k in range(10):
    # 좌표의 개수(동영상의 개수)만큼 반복
    for i in range(len_coord):
        _images = images[i]
        _len_images = len(_images)
        x = origin_coordinate[i][0]
        y = origin_coordinate[i][1]
        
        # 두 번째 차원의 리스트 초기화
        coord_list[i] = [[] for _ in range(_len_images)]

        img0 = _images[0] # 첫 번째 이미지를 target 이미지로 설정
        for j in range(_len_images):
            if j != _len_images - 1:
                img1 = _images[j+1]

                # LightGlue
                results_lightglue = match_lightglue(img0, img1)
                target_keypoint = results_lightglue["points0"].cpu().numpy()
                frame_keypoint = results_lightglue["points1"].cpu().numpy()
                if len(target_keypoint) < 6:
                    missing_inlier += 1

                homography, mask = CSRansac.csransac(target_keypoint, frame_keypoint)
                if mask == 0.3:
                    failed_inliers += 1
                projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)

                coord_list[i][j].append(projected_pts)
                
    #에러 측정            
    disappear_error = 0
    misannotate_error = 0
    pixel_error = 0

    for i in range(len_coord):
        origin_x = float_origin_coordinate[i][0]
        origin_y = float_origin_coordinate[i][1]
        
        for j in range(len(coord_list[i])-1):
            _coord = coord_list[i][j]
            
            x = _coord[0][0]
            y = _coord[0][1]
            
            x = x / 640
            y = y / 480
            
            x = round(x, 4)
            y = round(y, 4)
            
            # disappear_error
            if x < 0 or x > 1 or y < 0 or y > 1:
                disappear_error += 1
            
            distance = math.sqrt((origin_x - x)**2 + (origin_y - y)**2)
            
            # num_error
            if distance > 0.1:
                misannotate_error += 1
            
            # pixel_error
            if distance > pixel_error:
                pixel_error = distance
               
    print("disappear_error:", disappear_error)
    print("misannotate_error:", misannotate_error)
    print("pixel_error:", pixel_error)

    disappear_errors.append(disappear_error)
    misannotate_errors.append(misannotate_error)
    pixel_errors.append(pixel_error)
    

print("missing_inlier:", missing_inlier)
print("failed_inliers:", failed_inliers)

disappear_error: 272
misannotate_error: 870
pixel_error: 26.18959473200706
disappear_error: 274
misannotate_error: 855
pixel_error: 7.404622486908405
disappear_error: 259
misannotate_error: 853
pixel_error: 5.202331909718031
disappear_error: 271
misannotate_error: 837
pixel_error: 1.6864878338989968
disappear_error: 268
misannotate_error: 864
pixel_error: 1.4519503755257377
disappear_error: 261
misannotate_error: 834
pixel_error: 2.9066760920722077
disappear_error: 264
misannotate_error: 852
pixel_error: 10.110453093905798
disappear_error: 270
misannotate_error: 851
pixel_error: 2.1716921798017323
disappear_error: 259
misannotate_error: 890
pixel_error: 1.598593553505584
disappear_error: 276
misannotate_error: 850
pixel_error: 10.110453093905798
missing_inlier: 110
failed_inliers: 127


In [12]:
error1 = sum(disappear_errors) / len(disappear_errors)
error2 = sum(misannotate_errors) / len(misannotate_errors)
error3 = sum(pixel_errors) / len(pixel_errors)

print("disappear_error:", error1)
print("num_error:", error2)
print("pixel_error:", error3)

disappear_error: 267.4
num_error: 855.6
pixel_error: 6.883285535124935


In [20]:
#원본 이미지를 기준으로 호모그래피 행렬을 구하고, 호모그래피 행렬을 이용하여 특징점의 좌표를 변환하는 코드
len_coord = len(origin_coordinate)

coord_list = [[] for _i in range(len(origin_coordinate))]

disappear_errors = []
misannotate_errors = []
pixel_errors = []

missing_inlier = 0
failed_inliers = 0

# 10번 반복하여 측정한 에러를 구함
for k in range(10):
    # 좌표의 개수(동영상의 개수)만큼 반복
    for i in range(len_coord):
        _images = images[i]
        _len_images = len(_images)
        x = origin_coordinate[i][0]
        y = origin_coordinate[i][1]
        
        # 두 번째 차원의 리스트 초기화
        coord_list[i] = [[] for _ in range(_len_images)]

        img0 = _images[0] # 첫 번째 이미지를 target 이미지로 설정
        for j in range(_len_images):
            if j != _len_images - 1:
                img1 = _images[j+1]
                img1 = cv2.imread(img1)
                img1 = stabilizer.stabilize_frame(input_frame = img1)

                # LightGlue
                results_lightglue = match_lightglue(img0, img1)
                target_keypoint = results_lightglue["points0"].cpu().numpy()
                frame_keypoint = results_lightglue["points1"].cpu().numpy()
                if len(target_keypoint) < 6:
                    missing_inlier += 1

                homography, mask = CSRansac.csransac(target_keypoint, frame_keypoint)
                if mask == 0.3:
                    failed_inliers += 1
                projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)

                coord_list[i][j].append(projected_pts)
                
    #에러 측정            
    disappear_error = 0
    misannotate_error = 0
    pixel_error = 0

    for i in range(len_coord):
        origin_x = float_origin_coordinate[i][0]
        origin_y = float_origin_coordinate[i][1]
        
        for j in range(len(coord_list[i])-1):
            _coord = coord_list[i][j]
            
            x = _coord[0][0]
            y = _coord[0][1]
            
            x = x / 640
            y = y / 480
            
            x = round(x, 4)
            y = round(y, 4)
            
            # disappear_error
            if x < 0 or x > 1 or y < 0 or y > 1:
                disappear_error += 1
            
            distance = math.sqrt((origin_x - x)**2 + (origin_y - y)**2)
            
            # num_error
            if distance > 0.1:
                misannotate_error += 1
            
            # pixel_error
            if distance > pixel_error:
                pixel_error = distance
               
    print("disappear_error:", disappear_error)
    print("misannotate_error:", misannotate_error)
    print("pixel_error:", pixel_error)

    disappear_errors.append(disappear_error)
    misannotate_errors.append(misannotate_error)
    pixel_errors.append(pixel_error)
    

print("missing_inlier:", missing_inlier)
print("failed_inliers:", failed_inliers)

disappear_error: 260
misannotate_error: 881
pixel_error: 7.159248558571663
disappear_error: 275
misannotate_error: 849
pixel_error: 4.7647149059376215
disappear_error: 282
misannotate_error: 864
pixel_error: 7.159248558571663
disappear_error: 280
misannotate_error: 875
pixel_error: 39.0814013755959
disappear_error: 274
misannotate_error: 861
pixel_error: 4.7647149059376215
disappear_error: 301
misannotate_error: 868
pixel_error: 3.9299264323202805
disappear_error: 285
misannotate_error: 903
pixel_error: 12.300719351340005
disappear_error: 287
misannotate_error: 859
pixel_error: 4.7647149059376215
disappear_error: 276
misannotate_error: 858
pixel_error: 9.913376239991567
disappear_error: 275
misannotate_error: 854
pixel_error: 7.159248558571663
missing_inlier: 160
failed_inliers: 102


In [21]:
error1 = sum(disappear_errors) / len(disappear_errors)
error2 = sum(misannotate_errors) / len(misannotate_errors)
error3 = sum(pixel_errors) / len(pixel_errors)

print("disappear_error:", error1)
print("num_error:", error2)
print("pixel_error:", error3)

disappear_error: 279.5
num_error: 867.2
pixel_error: 10.09973137927756


## check speed

In [ ]:
# video_frames 폴더에서 프레임 파일 리스트 가져오기
video_frames = os.listdir('video')

# 프레임 수 초기화
frame_count = 0

# 프레임 별 처리 시간 리스트 초기화
frame_processing_times = []

x = 637 // 2
y = 367 // 2

image0 = load_image_from_path("img1.png", grayscale=True)
cap = cv2.VideoCapture('demo_video_resized.mp4')

# 각 프레임 처리
while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    start_time = time.time()
    
    feats0 = extractor.extract(image0.to(device))
    image1 = stabilizer.stabilize_frame(input_frame = frame)
    image1 = load_image(frame, grayscale=True)
    feats1 = extractor.extract(image1.to(device))
    
    matches01 = matcher({"image0": feats0, "image1": feats1})
    
    feats0, feats1, matches01 = [
        rbd(x) for x in [feats0, feats1, matches01]
    ]  # remove batch dimension
    
    kpts0, kpts1, matches = feats0["keypoints"], feats1["keypoints"], matches01["matches"]
    m_kpts0, m_kpts1 = kpts0[matches[..., 0]], kpts1[matches[..., 1]]
    
    homography, _ = CSRansac.csransac(m_kpts0.cpu().numpy(), m_kpts1.cpu().numpy())
    projected_pts = CSRansac.perspective_transform(np.array([x, y]), homography)
    
    cv2.circle(frame, (int(projected_pts[0]), int(projected_pts[1])), 5, (0, 0, 255), -1)
    
    # 현재 시간 측정
    current_time = time.time()
    
    # 프레임 처리 시간 계산
    frame_processing_time = current_time - start_time
    frame_processing_times.append(frame_processing_time)
    
    # 이전 프레임 처리 시간 업데이트
    prev_frame_time = current_time

    # FPS 계산
    fps = 1.0 / frame_processing_time

    # 프레임 수 증가
    frame_count += 1

    
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(5)
    if key == 27:
        break
    
    
cap.release()
cv2.destroyAllWindows()

# 전체 처리 시간 계산
total_processing_time = sum(frame_processing_times)

# 전체 프레임 수와 전체 처리 시간을 사용하여 평균 FPS 계산
average_fps = frame_count / total_processing_time

print(f"Total Frames Processed: {frame_count}")
print(f"Average FPS: {average_fps:.2f}")

In [35]:
from vidstab import VidStab

# Using defaults
stabilizer = VidStab()
stabilizer.stabilize(input_path='demo_video_resized.mp4', output_path='stable_demo_video.mp4')